In [1]:
import os
os.chdir('../')

import warnings
warnings.filterwarnings('ignore')

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os

!nvidia-smi
os.environ["CUDA_VISIBLE_DEVICES"]="4"

Tue Jul 11 21:19:18 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.125.06   Driver Version: 525.125.06   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A5000    Off  | 00000000:1B:00.0 Off |                    0 |
| 30%   28C    P8    13W / 230W |   1836MiB / 23028MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A5000    Off  | 00000000:1C:00.0 Off |                  Off |
| 30%   

### Hyperparams

In [3]:
n_mels = 80
n_outputs = 61
n_frames = 400

### Model

In [4]:
from model.model_transformer_reg import Model
from utils.util import *
from tensorboardX import SummaryWriter

step = 0
device = 'cuda:0'

# Model
model = Model(in_dim=n_mels, h_dim=512, out_dim=n_outputs)
model = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
print('done')

done


In [5]:
# warm start
checkpoint = torch.load('/data/scpark/save/lips/train06.23-1/save_120000', map_location=torch.device('cpu'))
model_state_dict = model.state_dict()

for key in checkpoint['model_state_dict']:
    if key in model_state_dict.keys():
        if checkpoint['model_state_dict'][key].shape == model_state_dict[key].shape:
            model_state_dict[key] = checkpoint['model_state_dict'][key]
            print(key)
model.load_state_dict(model_state_dict, strict=True)
print('warm start')

prenet.bias
encoder.attn_layers.0.emb_rel_k
encoder.attn_layers.0.emb_rel_v
encoder.attn_layers.0.conv_q.weight
encoder.attn_layers.0.conv_q.bias
encoder.attn_layers.0.conv_k.weight
encoder.attn_layers.0.conv_k.bias
encoder.attn_layers.0.conv_v.weight
encoder.attn_layers.0.conv_v.bias
encoder.attn_layers.0.conv_o.weight
encoder.attn_layers.0.conv_o.bias
encoder.attn_layers.1.emb_rel_k
encoder.attn_layers.1.emb_rel_v
encoder.attn_layers.1.conv_q.weight
encoder.attn_layers.1.conv_q.bias
encoder.attn_layers.1.conv_k.weight
encoder.attn_layers.1.conv_k.bias
encoder.attn_layers.1.conv_v.weight
encoder.attn_layers.1.conv_v.bias
encoder.attn_layers.1.conv_o.weight
encoder.attn_layers.1.conv_o.bias
encoder.attn_layers.2.emb_rel_k
encoder.attn_layers.2.emb_rel_v
encoder.attn_layers.2.conv_q.weight
encoder.attn_layers.2.conv_q.bias
encoder.attn_layers.2.conv_k.weight
encoder.attn_layers.2.conv_k.bias
encoder.attn_layers.2.conv_v.weight
encoder.attn_layers.2.conv_v.bias
encoder.attn_layers.2.conv

### Load

In [6]:
save_dir = '/data/scpark/save/lips/train07.11-1/'
!mkdir -p $save_dir
!ls -lt $save_dir

writer = SummaryWriter(save_dir)

if False:
    step, model, _, optimizer = load(save_dir, 56389, model, None, optimizer)

total 15100
-rw-rw-r-- 1 scpark scpark 15456205  7월 11 21:18 save_0
-rw-rw-r-- 1 scpark scpark      131  7월 11 21:17 events.out.tfevents.1689077737.GPUSVR01


### Dataset

In [7]:
import os
from data.arkit_dataset import LipsDataset, CombinedDataset, CombinedCollate

root_dir = '/data/speech/digital_human/preprocessed/'
files = sorted([os.path.join(root_dir, file) for file in os.listdir(root_dir)])
print(len(files))

train_datasets = []
test_datasets = []

for file in files:
    print(file)
    dataset = LipsDataset(file, n_mels, n_frames)
    if '_10_' in file:
        test_datasets.append(dataset)
    else:
        train_datasets.append(dataset)
print(len(train_datasets), len(test_datasets))

39
/data/speech/digital_human/preprocessed/MH_ARKit_001_1_iPhone_raw.npy
/data/speech/digital_human/preprocessed/MH_ARKit_001_2_iPhone_raw.npy
/data/speech/digital_human/preprocessed/MH_ARKit_001_3_iPhone_raw.npy
/data/speech/digital_human/preprocessed/MH_ARKit_001_4_iPhone_raw.npy
/data/speech/digital_human/preprocessed/MH_ARKit_001_5_iPhone_raw.npy
/data/speech/digital_human/preprocessed/MH_ARKit_001_6_iPhone_raw.npy
/data/speech/digital_human/preprocessed/MH_ARKit_001_7_iPhone_raw.npy
/data/speech/digital_human/preprocessed/MH_ARKit_001_8_iPhone_raw.npy
/data/speech/digital_human/preprocessed/MH_ARKit_001_9_iPhone_raw.npy
/data/speech/digital_human/preprocessed/MH_ARKit_002_10_iPhone_raw.npy
/data/speech/digital_human/preprocessed/MH_ARKit_002_1_iPhone_raw.npy
/data/speech/digital_human/preprocessed/MH_ARKit_002_2_iPhone_raw.npy
/data/speech/digital_human/preprocessed/MH_ARKit_002_3_iPhone_raw.npy
/data/speech/digital_human/preprocessed/MH_ARKit_002_4_iPhone_raw.npy
/data/speech/dig

In [8]:
train_loader = torch.utils.data.DataLoader(CombinedDataset(train_datasets), 
                                           num_workers=16, shuffle=True, batch_size=32, collate_fn=CombinedCollate())
test_loader = torch.utils.data.DataLoader(CombinedDataset(test_datasets), 
                                          num_workers=10, shuffle=True, batch_size=10, collate_fn=CombinedCollate())
print('done')

done


In [ ]:
from IPython import display
import librosa.display
import matplotlib.pyplot as plt

isnan = False
while True:
    if isnan:
        break
    for batch in train_loader:
        inputs = torch.Tensor(batch['mel']).transpose(1, 2).to(device)
        targets = torch.Tensor(batch['blend']).transpose(1, 2).to(device)
        
        model.train()
        model.zero_grad()
        outputs = model(inputs, targets)
        
        print(step)
        loss = 0
        for key in outputs.keys():
            if 'loss' in key:
                loss += outputs[key]
                print(key, outputs[key].item())
        if torch.isnan(loss):
            isnan = True
            break
        loss.backward()
        optimizer.step()
        print(step, loss.item())
        
        if step % 1000 == 0:
            writer.add_scalar('train_loss', loss.item(), step)
        
        if step % 1000 == 0:
            display.clear_output()
            
            losses = []
            for i, batch in enumerate(test_loader):
                if i >= 10:
                    break
                    
                inputs = torch.Tensor(batch['mel']).transpose(1, 2).to(device)
                targets = torch.Tensor(batch['blend']).transpose(1, 2).to(device)

                model.eval()
                with torch.no_grad():
                    outputs = model(inputs, targets)
                    
                loss = 0
                for key in outputs.keys():
                    if 'loss' in key:
                        loss += outputs[key]
                print('test :', i, loss.item())
                losses.append(loss)        
            
            test_loss = torch.stack(losses).mean().item()
            print('test_loss :', test_loss)
            writer.add_scalar('test_loss', test_loss, step)
            
#             plt.figure(figsize=[18, 4])
#             librosa.display.specshow(targets[0].data.cpu().numpy(), cmap='magma')
#             plt.show()

#             plt.figure(figsize=[18, 4])
#             librosa.display.specshow(outputs['y_pred'][0].data.cpu().numpy(), cmap='magma')
#             plt.show()
            
#             for i in [20, 37]:
#                 plt.figure(figsize=[18, 2])
#                 plt.title(str(i))
#                 plt.plot(targets[0].data.cpu().numpy()[i])
#                 plt.plot(outputs['y_pred'][0].data.cpu().numpy()[i])
#                 plt.show()

        if step % 1000 == 0:
            save(save_dir, step, model, None, optimizer)
    
        step += 1

7982
loss 0.012133250944316387
7982 0.012133250944316387
7983
loss 0.011064130812883377
7983 0.011064130812883377
7984
loss 0.01085596065968275
7984 0.01085596065968275
7985
loss 0.011232595890760422
7985 0.011232595890760422
7986
loss 0.012047233991324902
7986 0.012047233991324902
7987
loss 0.011724227108061314
7987 0.011724227108061314
7988
loss 0.01215800829231739
7988 0.01215800829231739
7989
loss 0.011444835923612118
7989 0.011444835923612118
7990
loss 0.011664369143545628
7990 0.011664369143545628
7991
loss 0.01078946702182293
7991 0.01078946702182293
7992
loss 0.01187424547970295
7992 0.01187424547970295
7993
loss 0.011195674538612366
7993 0.011195674538612366
7994
loss 0.0120087256655097
7994 0.0120087256655097
7995
loss 0.011413974687457085
7995 0.011413974687457085
7996
loss 0.011678694747388363
7996 0.011678694747388363
7997
loss 0.011530161835253239
7997 0.011530161835253239


In [10]:
save(save_dir, step, model, None, optimizer)

saved /data/scpark/save/lips/train07.11-1/save_7982


In [ ]:
print('done')